# Prédire la demande d’un article donné

## Objectif

Prédire la quantité vendue (`Quantity`) d’un article à un instant futur (par exemple, à la prochaine heure).

Chaque observation agrégée correspond à la quantité totale vendue d’un article sur une période donnée (heure, jour, etc.).

---

## Données utilisées

Variables disponibles dans le jeu de données initial :

source
# Compter les ventes par article, puis trier par nom d'article (ordre alphabétique)
ventes_par_article = data["article"].value_counts().sort_index()

# Liste de tous les articles par ordre alphabétique avec leur nombre vendu
print("Articles (ordre alphabétique) et nombre vendus :")
print(ventes_par_article)

# Aperçu des 10 articles les plus vendus (triés par nombre)
top10 = ventes_par_article.sort_values(ascending=False).head(10)
print("\nTop 10 articles les plus vendus (par nombre) :")
print(top10)
### Import du dataset

In [1]:
# importer le dataset
import pandas as pd
data = pd.read_csv('data_tmp/bakery_sales.csv')
data.head()

,Unnamed: 0,date,time,ticket_number,article,Quantity,unit_price
0,0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €"
1,1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €"
2,4,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €"
3,5,2021-01-02,09:14,150041.0,PAIN,1.0,"1,15 €"
4,8,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €"


# 1. Nettoyage et exploration initiale des données de ventes

## Objectif

1. Vérifier quels articles ont été très peu vendus (peu d’occurrences dans le dataset).  
2. Afficher la liste complète des articles uniques, triés par ordre alphabétique.

## A. Vérification des colonnes disponibles et des valeurs manquantes
CONCLUSION : Rien de manquant

In [2]:
# Vérifier les colonnes et types de données
print(data.info())

# Vérifier les valeurs manquantes
# AUCUNE

# Vérifier s'il y a des doublons exacts
nb_duplicated = data.duplicated().sum()
print(f"Nombre de lignes dupliquées : {nb_duplicated}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     234005 non-null  int64  
 1   date           234005 non-null  object 
 2   time           234005 non-null  object 
 3   ticket_number  234005 non-null  float64
 4   article        234005 non-null  object 
 5   Quantity       234005 non-null  float64
 6   unit_price     234005 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 12.5+ MB
None
Nombre de lignes dupliquées : 0


## B. Compter le nombre de ventes par article, pour supprimer quelques valeurs "Article" inutiles

1) Regrouper des articles entre eux ( ex: "baguette apéro" et "baguette" dans la même valeur "baguette")
2) On trie les articles du moins vendu au plus vendu, puis on prend les premiers tant que la somme cumulée reste ≤ 1 % des ventes totales. On supprime ceux là

Nombre d'articles uniques avant modification :
Nombre d'articles uniques après regroupement :
Nombre d'articles uniques après suppression :

### Afficher la liste des noms d'articles et leur vente total pour faire la selection des modifications

In [3]:
# Nombre total de lignes par article
ventes_par_article = data["article"].value_counts()
nombre_articles_uniques = len(ventes_par_article)
print(f"Nombre d'articles uniques avant modification : {nombre_articles_uniques}")

#Trier les articles par ordre alphabétique
ventes_par_article = ventes_par_article.sort_index()

#Afficher tous les articles avec leur nombre de ventes
print(ventes_par_article.to_string())

Nombre d'articles uniques avant modification : 149
article
.                               5
12 MACARON                     65
ARMORICAIN                      2
ARTICLE 295                     1
BAGUETTE                    15292
BAGUETTE APERO                 73
BAGUETTE GRAINE              1505
BANETTE                     15130
BANETTINE                    2817
BOISSON 33CL                 1481
BOTTEREAU                      85
BOULE 200G                   2691
BOULE 400G                   4099
BOULE POLKA                   502
BRIOCHE                      1657
BRIOCHE DE NOEL                19
BRIOCHETTE                     46
BROWNIES                       38
BUCHE 4PERS                    10
BUCHE 6PERS                     8
BUCHE 8PERS                     1
CAFE OU EAU                  1436
CAKE                            1
CAMPAGNE                     3905
CARAMEL NOIX                   69
CEREAL BAGUETTE              4961
CHAUSSON AUX POMMES          1442
CHOCOLAT               

### Regroupement proposés
BAGUETTE APERO                 73 --> BAGUETTE
BRIOCHE DE NOEL                19 --> BRIOCHE
BRIOCHETTE                     46 --> BRIOCHE
DEMI PAIN                     163 --> DEMI PAIN
ECLAIR FRAISE PISTACHE         16 --> ECLAIR
FINANCIER                      17 --> FINANCIER X5
GD FAR BRETON                 161 --> GRAND FAR BRETON
PAIN GRAINES                    3 --> PAIN
PAIN NOIR                       1 --> PAIN
PAIN S/SEL                    131 --> PAIN
PAIN SUISSE PEPITO             18 --> PAIN
ROYAL 4P                       61 --> ROYAL
ROYAL 6P                       62 --> ROYAL
SACHET DE VIENNOISERIE          1 --> SACHET VIENNOISERIE
SAND JB                         5 --> SAND JB EMMENTAL
SPECIAL BREAD KG              460 --> SPECIAL BREAD
TARTELETTE CHOC                52 --> TARTELETTE
TARTELETTE COCKTAIL             1 --> TARTELETTE
TARTELETTE FRAISE             493 --> TARTELETTE
TROPEZIENNE FRAMBOISE          26 --> TROPEZIENNE


### Création (merger des articles différents dans un nouvel article)
GAL FRANGIPANE 4P             203 --> GALETTE FRANGIPANE 
GAL FRANGIPANE 6P             122 --> GALETTE FRANGIPANE

GAL POIRE CHOCO 4P             24 --> GALETTE FRUIT
GAL POIRE CHOCO 6P             15 --> GALETTE FRUIT
GAL POMME 4P                   97 --> GALETTE FRUIT
GAL POMME 6P                   61 --> GALETTE FRUIT
GALETTE 8 PERS                  2 --> GALETTE FRUIT

PLAT                            8 --> PLAT PREPARE
PLAT 6.50E                      2 --> PLAT PREPARE
PLAT 7.00                      94 --> PLAT PREPARE
PLAT 7.60E                    244 --> PLAT PREPARE
PLAT 8.30E                    116 --> PLAT PREPARE
PLATPREPARE5,50                 3 --> PLAT PREPARE
PLATPREPARE6,00                 1 --> PLAT PREPARE
PLATPREPARE6,50                 6 --> PLAT PREPARE
PLATPREPARE7,00                11 --> PLAT PREPARE

TARTE FINE                     61 --> TARTE FRUITS
TARTE FRAISE 4PER             149 --> TARTE FRUITS
TARTE FRAISE 6P               125 --> TARTE FRUITS
TARTE FRUITS 4P               358 --> TARTE FRUITS
TARTE FRUITS 6P               272 --> TARTE FRUITS

### On trie les articles du moins vendu au plus vendu, puis on prend les premiers tant que la somme cumulée reste ≤ 1 % des ventes totales.

In [4]:

# Comptage des ventes par article (si ce n'est pas déjà fait)
ventes_par_article = data["article"].value_counts()

# Connaitre les articles dont le cumul représente au plus 1% des ventes totales
ventes_totales = ventes_par_article.sum()
seuil_1pct_ventes = ventes_totales * 0.01

print(f"Seuil pour 1% des ventes totales : {seuil_1pct_ventes:.0f} ventes")

# Trier les articles du moins vendu au plus vendu
ventes_tries = ventes_par_article.sort_values(ascending=True)

# Somme cumulée des ventes en partant du moins vendu
cumul_ventes = ventes_tries.cumsum()

# Garder les articles tant que le cumul reste <= 1% des ventes totales
articles_sous_1pct = ventes_tries[cumul_ventes <= seuil_1pct_ventes]

print("Articles les moins vendus dont les ventes cumulées représentent au plus 1% des ventes totales :")
print(articles_sous_1pct.to_string())

print(f"\nTotal cumulé de ces articles : {articles_sous_1pct.sum()} ventes")
print(f"Ce qui représente {articles_sous_1pct.sum() / ventes_totales * 100:.3f}% des ventes totales.")

Seuil pour 1% des ventes totales : 2340 ventes
Articles les moins vendus dont les ventes cumulées représentent au plus 1% des ventes totales :
article
SACHET DE VIENNOISERIE        1
PLAQUE TARTE 25P              1
TARTELETTE COCKTAIL           1
ARTICLE 295                   1
CAKE                          1
TROIS CHOCOLAT                1
DOUCEUR D HIVER               1
REDUCTION SUCREES 24          1
CRUMBLECARAMEL OU PISTAE      1
PLATPREPARE6,00               1
PAIN NOIR                     1
BUCHE 8PERS                   1
ARMORICAIN                    2
FORMULE PATE                  2
PATES                         2
PLAT 6.50E                    2
GD PLATEAU SALE               2
GALETTE 8 PERS                2
PLATPREPARE5,50               3
PT PLATEAU SALE               3
TULIPE                        3
PAIN GRAINES                  3
GUERANDAIS                    3
SAND JB                       5
.                             5
PALMIER                       5
PLATPREPARE6,50  

### Suppressions proposées (vente <=10)
ARMORICAIN                      2
ARTICLE 295                     1
BROWNIES                       38
BUCHE 4PERS                    10
BUCHE 6PERS                     8
BUCHE 8PERS                     1
CAKE                            1
RUMBLE                          7
CRUMBLECARAMEL OU PISTAE        1
DELICETROPICAL                 41
DOUCEUR D HIVER                 1
ENTREMETS                      24
FORMULE PATE                    2
FORMULE PLAT PREPARE           10
GUERANDAIS                      3
MERINGUE                       51
NID DE POULE                   12
PALMIER                         5
PATES                           2
PLAQUE TARTE 25P                1
PT PLATEAU SALE                 3
REDUCTION SUCREES 12            8
REDUCTION SUCREES 24            1
THE                             6
TRIANGLES                      67
TROIS CHOCOLAT                  1
TULIPE                          3

In [10]:
df = pd.read_csv('data_tmp/bakery_sales_reduit.csv').drop(columns=['Unnamed: 0'])
df.head()

,date,time,ticket_number,article,Quantity,unit_price
0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €"
1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €"
2,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €"
3,2021-01-02,09:14,150041.0,PAIN,1.0,"1,15 €"
4,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €"
